## ROI analysis of the Agency task 

This code performs a ROI analysis on the contrast maps in the rTPJ, lTPJ, SMA, dlPFC and IC ROIs of n = 18 participants. The con values are taken from the turbulence > baseline(non-turbulence) contrast for each of the two sessions (pre and post NF training)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import sys
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp
import statsmodels.api as sm
import statsmodels as sts
import statsmodels.stats.multitest as smm
import seaborn as sns
import pingouin as pg
import matplotlib
from matplotlib.ticker import MaxNLocator



In [ ]:
# Read in excel files. Get BOLD signal values for ROI analysis (run-wise contrast values)
# From the turbulence > baseline contrast

# rTPJ

# Define the path to the Excel file -> choose either the data from the individual masks or full masks 
file_path_rTPJ = './rTPJ_ROI.xlsx'

# Read the sheet with mean BOLD signal per run (or the Percent signal change for the gamephase contrast)
mean_rTPJ = pd.read_excel(file_path_rTPJ, sheet_name='Mean_Values')

# Leave out last column (Run 9 - Run 1 difference column)
mean_rTPJ = mean_rTPJ.iloc[:,0:3]

# Preprocess the dataframe
mean_rTPJ.rename(columns={mean_rTPJ.columns[0]: 'P-Code'}, inplace=True)
mean_rTPJ.set_index('P-Code', inplace=True)

# lTPJ
file_path_lTPJ = './rTPJ_ROI.xlsx'
mean_lTPJ = pd.read_excel(file_path_lTPJ)

# Leave out last column (Run 9 - Run 1 difference column)
mean_lTPJ = mean_lTPJ.iloc[:,0:3]

# Preprocess the dataframe
mean_lTPJ.rename(columns={mean_lTPJ.columns[0]: 'P-Code'}, inplace=True)
mean_lTPJ.set_index('P-Code', inplace=True)

In [ ]:
# SMA, dlFPC, IC

# Glasser atlas the first 180 regions are from the left hemisphere, the last 180 regions are from the right hemisphere
file_path_all = './mean_contrast_values_turb_base_Glasser_atlas.xlsx'

# Read the sheet with mean contrast values for each region of the atlas
con_sess1 = pd.read_excel(file_path_all, sheet_name='Sess1')
con_sess2 = pd.read_excel(file_path_all, sheet_name='Sess2')

# Preprocess the dataframe

con_sess1.set_index('P-Code', inplace=True)
con_sess2.set_index('P-Code', inplace=True)

In [ ]:
# Define a mapping of ROI names to column lists
roi_map = {
    'Left insula': ['ROI_106', 'ROI_167'],
    'Left SMA': ['ROI_ 44', 'ROI_ 55'],
    'Left DLPFC': ['ROI_ 83', 'ROI_ 84', 'ROI_ 86'],
    'Right insula': ['ROI_286', 'ROI_347'],
    'Right SMA': ['ROI_224', 'ROI_235'],
    'Right DLPFC': ['ROI_263', 'ROI_264', 'ROI_266'],
    'Insula': ['ROI_106', 'ROI_167', 'ROI_286', 'ROI_347'],
    'SMA': ['ROI_ 44', 'ROI_ 55', 'ROI_224', 'ROI_235'],
    'DLPFC': ['ROI_ 83', 'ROI_ 84', 'ROI_ 86', 'ROI_263', 'ROI_264', 'ROI_266']
}

# Function to create session ROI DataFrame
def create_sess_ROI(con_sess, mean_rTPJ, mean_lTPJ, session_label):
    data = {
        'Right TPJ': mean_rTPJ[session_label],
        'Left TPJ': mean_lTPJ[session_label]
    }
    
    for roi_name, cols in roi_map.items():
        data[roi_name] = np.mean(con_sess[cols], axis=1)
    
    return pd.DataFrame(data)

# Create session DataFrames
sess1_ROI = create_sess_ROI(con_sess1, mean_rTPJ, mean_lTPJ, 'Session 1')
sess2_ROI = create_sess_ROI(con_sess2, mean_rTPJ, mean_lTPJ, 'Session 2')

# Save as excel file

output_path = "./Agency_SoA_network_analysis.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    sess1_ROI.to_excel(writer, sheet_name='Sess1', index=True)
    sess2_ROI.to_excel(writer, sheet_name='Sess2', index=True)



In [ ]:
# Define learners(responders) and non-learners(non-responders) based on behavioral data (SoA ratings across runs)
sess1_ROI['learners'] = ['False','False','False','True','False','False','True','False','False','True','True','True','True','False','True','True','False','False']

SoA_learners_sess1 = sess1_ROI[sess1_ROI['learners']=='True']
SoA_learners_sess2 = sess2_ROI[sess1_ROI['learners']=='True']

SoA_nonlearners_sess1 = sess1_ROI[sess1_ROI['learners']=='False']
SoA_nonlearners_sess2 = sess2_ROI[sess1_ROI['learners']=='False']

In [ ]:
# Comparison of Sess1 & Sess2 for ROIs in the sense of agency network 

who = 1  # 0 = all subjects, 1 = learners, 2 = non-learners

# Select the dataframe based on the group
dfs = {
    0: (sess1_ROI, sess2_ROI),
    1: (SoA_learners_sess1, SoA_learners_sess2),
    2: (SoA_nonlearners_sess1, SoA_nonlearners_sess2)
}

df_1, df_2 = dfs[who]

# List of ROIs to compare
rois = ['Right TPJ', 'Left TPJ', 'Left insula', 'Right insula', 'Left SMA', 'Right SMA', 'Left DLPFC', 'Right DLPFC']

tvalues = [0]*len(rois)
pvalues = [0]*len(rois)

# Loop over ROIs
for i, roi in enumerate(rois):
    sess1_data = df_1[roi]
    sess2_data = df_2[roi]
    tvalues[i], pvalues[i] = ttest_rel(sess1_data, sess2_data)
    print(f'{roi} Sess 1 vs. Sess 2.\n t-value: {tvalues[i]}, p-value: {pvalues[i]}')

# FDR correction
_, p_corrected = smm.fdrcorrection(pvalues, alpha=0.05)
print('FDR-corrected p-values:', p_corrected)


In [ ]:
# Comparison of Sess1 & Sess2 for ROIs in the sense of agency network - both hemispheres

who = 1  # 0 = all subjects, 1 = learners, 2 = non-learners

# Select the dataframe based on the group
dfs = {
    0: (sess1_ROI, sess2_ROI),
    1: (SoA_learners_sess1, SoA_learners_sess2),
    2: (SoA_nonlearners_sess1, SoA_nonlearners_sess2)
}

df_1, df_2 = dfs[who]

# ROIs for whole-hemisphere comparison
rois = ['Insula', 'SMA', 'DLPFC']

tvalues2 = [0]*len(rois)
pvalues2 = [0]*len(rois)

# Loop over ROIs
for i, roi in enumerate(rois):
    sess1_data = df_1[roi]
    sess2_data = df_2[roi]
    tvalues2[i], pvalues2[i] = ttest_rel(sess1_data, sess2_data)
    print(f'{roi} Sess 1 vs. Sess 2.\n t-value: {tvalues2[i]}, p-value: {pvalues2[i]}')

# FDR correction
_, p_corrected = smm.fdrcorrection(pvalues2, alpha=0.05)
print('FDR-corrected p-values:', p_corrected)

In [ ]:
# Violin plot

# Choose ROI
which_ROI = 'DLPFC'

# Map ROI names to data sources
roi_map = {
    'rTPJ': lambda: pd.DataFrame({'Session 1': mean_rTPJ['Session 1'], 'Session 2': mean_rTPJ['Session 2']}),
    'lTPJ': lambda: pd.DataFrame({'Session 1': mean_lTPJ['Session 1'], 'Session 2': mean_lTPJ['Session 2']}),
    'IC': lambda: pd.DataFrame({'Session 1': sess1_ROI['Insula'], 'Session 2': sess2_ROI['Insula']}),
    'SMA': lambda: pd.DataFrame({'Session 1': sess1_ROI['SMA'], 'Session 2': sess2_ROI['SMA']}),
    'DLPFC': lambda: pd.DataFrame({'Session 1': sess1_ROI['DLPFC'], 'Session 2': sess2_ROI['DLPFC']})
}

df = roi_map[which_ROI]()

# Assign learners
learners_list = ['False','False','False','True','False','False','True','False','False','True',
                 'True','True','True','False','True','True','False','False']
df['learners'] = learners_list

# Split by learners/non-learners
groups = {
    'Responder': df[df['learners']=='True'],
    'Non-Responder': df[df['learners']=='False']
}

# Prepare long-format dataframe for violin plot
df_vp = pd.concat([
    pd.DataFrame({
        'Value': g['Session 1'],
        'Session': 'Pre-NF',
        'Group': name
    }).append(pd.DataFrame({
        'Value': g['Session 2'],
        'Session': 'Post-NF',
        'Group': name
    }))
    for name, g in groups.items()
], ignore_index=True)

# Assign Subject index
df_vp.index = np.arange(1, len(df_vp)+1)
df_vp.index.name = 'Subject'

# Plot settings
plt.style.use('default')
plt.figure(dpi=120)
ax = sns.violinplot(
    x='Group',
    y='Value',
    data=df_vp,
    hue='Session',
    palette={'Pre-NF': 'white', 'Post-NF': 'darkgrey'},
    cut=0.7,
    inner="box",
    linewidth=1.2,
    width=0.6
)

# Session styles for violin edges
session_styles = {
    'Pre-NF': {'color': '#99E9FF', 'linewidth': 1.8},
    'Post-NF': {'color': 'black', 'linewidth': 1.5}
}

for i, session in enumerate(['Pre-NF','Post-NF']*len(groups)):
    artist = ax.collections[i]
    if isinstance(artist, matplotlib.collections.PolyCollection):
        artist.set_edgecolor(session_styles[session]['color'])
        artist.set_linewidth(session_styles[session]['linewidth'])

# Connect points for each subject
for _, sub in df_vp.groupby('Subject'):
    ax.plot(sub['Group'], sub['Value'], color='dimgray', alpha=0.6, linewidth=0.9)

# Labels and styling
plt.ylabel('Contrast estimate', size=15)
plt.xlabel(' ', size=17)
plt.xticks([0,1], size=18, labels=['Resp.', 'Non-Resp.'])
sns.despine(top=True, right=True)
plt.title(which_ROI, size=30, pad=10)
plt.yticks(size=15)
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))

# Custom legend
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles[:2], labels[:2], title='Session', bbox_to_anchor=(1.05,1), loc='upper left')

plt.tight_layout()
plt.show()